In [6]:
!pip install --no-index /kaggle/input/datasets/kurshidbasheer/biopython-offline/biopython-1.83-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


Processing /kaggle/input/datasets/kurshidbasheer/biopython-offline/biopython-1.83-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
biopython is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [7]:
!pip install --no-index /kaggle/input/datasets/kurshidbasheer/pyg-2-7-torch-2-9-cpu-py312-kur/torch_geometric-2.7.0-py3-none-any.whl

Processing /kaggle/input/datasets/kurshidbasheer/pyg-2-7-torch-2-9-cpu-py312-kur/torch_geometric-2.7.0-py3-none-any.whl
torch-geometric is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [9]:

import torch
import torch_geometric
from torch_geometric.nn import GCNConv

print("Torch:", torch.__version__)
print("PyG:", torch_geometric.__version__)

# Small forward pass test
x = torch.randn(4, 8)
edge_index = torch.tensor([[0,1,2,3],
                           [1,0,3,2]])

conv = GCNConv(8, 16)
out = conv(x, edge_index)

print("Output shape:", out.shape)


Torch: 2.9.0+cpu
PyG: 2.7.0
Output shape: torch.Size([4, 16])
